# Import libraries

In [23]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np


# Import file

In [24]:
spaceLaunch_df = pd.read_csv("Space_Corrected.csv")
spaceLaunch_df.dtypes

Unnamed: 0         int64
Unnamed: 0.1       int64
Company Name      object
Location          object
Datum             object
Detail            object
Status Rocket     object
 Rocket           object
Status Mission    object
dtype: object

# Data Preprocessing

In [25]:
#Bỏ đi 2 cột Unnamed: 0 và Unnamed: 0.1 vì không cần thiết
del spaceLaunch_df["Unnamed: 0"]
del spaceLaunch_df["Unnamed: 0.1"]

#Convert cột rocket sang kiểu Float
spaceLaunch_df[" Rocket"] = spaceLaunch_df[" Rocket"].astype(str)
for i in range(len(spaceLaunch_df[" Rocket"])):
    spaceLaunch_df[" Rocket"][i] = spaceLaunch_df[" Rocket"][i].strip(", ")
    spaceLaunch_df[" Rocket"][i] = spaceLaunch_df[" Rocket"][i].replace(",","")
spaceLaunch_df[" Rocket"] = spaceLaunch_df[" Rocket"].astype(float)

#Tên cột Rocket bị dư một dấu cách phía trước, không để ý có thể gây lỗi nên cần được bỏ dấu cách đi
spaceLaunch_df = spaceLaunch_df.rename(columns= {" Rocket" : "Rocket"})

#Cột Datum là kiểu dữ liệu object cần được chuyển sang dạng datetime
spaceLaunch_df["Datum"] = pd.to_datetime(spaceLaunch_df["Datum"])

#Tạo thêm cột year
spaceLaunch_df["Year"] = spaceLaunch_df["Datum"].apply(lambda datetime: datetime.year)

spaceLaunch_df

,Company Name,Location,Datum,Detail,Status Rocket,Rocket,Status Mission,Year
0,SpaceX,"LC-39A, Kennedy Space Center, Florida, USA",2020-08-07 05:12:00+00:00,Falcon 9 Block 5 | Starlink V1 L9 & BlackSky,StatusActive,50.00,Success,2020
1,CASC,"Site 9401 (SLS-2), Jiuquan Satellite Launch Ce...",2020-08-06 04:01:00+00:00,Long March 2D | Gaofen-9 04 & Q-SAT,StatusActive,29.75,Success,2020
2,SpaceX,"Pad A, Boca Chica, Texas, USA",2020-08-04 23:57:00+00:00,Starship Prototype | 150 Meter Hop,StatusActive,NaN,Success,2020
3,Roscosmos,"Site 200/39, Baikonur Cosmodrome, Kazakhstan",2020-07-30 21:25:00+00:00,Proton-M/Briz-M | Ekspress-80 & Ekspress-103,StatusActive,65.00,Success,2020
4,ULA,"SLC-41, Cape Canaveral AFS, Florida, USA",2020-07-30 11:50:00+00:00,Atlas V 541 | Perseverance,StatusActive,145.00,Success,2020
...,...,...,...,...,...,...,...,...
4319,US Navy,"LC-18A, Cape Canaveral AFS, Florida, USA",1958-02-05 07:33:00+00:00,Vanguard | Vanguard TV3BU,StatusRetired,NaN,Failure,1958
4320,AMBA,"LC-26A, Cape Canaveral AFS, Florida, USA",1958-02-01 03:48:00+00:00,Juno I | Explorer 1,StatusRetired,NaN,Success,1958
4321,US Navy,"LC-18A, Cape Canaveral AFS, Florida, USA",1957-12-06 16:44:00+00:00,Vanguard | Vanguard TV3,StatusRetired,NaN,Failure,1957
4322,RVSN USSR,"Site 1/5, Baikonur Cosmodrome, Kazakhstan",1957-11-03 02:30:00+00:00,Sputnik 8K71PS | Sputnik-2,StatusRetired,NaN,Success,1957


# Analysis

## Question 1

In [96]:
space_company_5_df = spaceLaunch_df[spaceLaunch_df["Year"] > 2015]
space_company_5_df = space_company_5_df[~np.isnan(space_company_5_df["Rocket"])]

company_success_launch_df = space_company_5_df[space_company_5_df["Status Mission"] == "Success"].groupby(["Year","Company Name"])["Status Mission"].count().sort_values(ascending = False) 
company_launch_df = space_company_5_df.groupby("Year","Company Name")["Status Mission"].count().sort_values(ascending = False)

company_success_launch_ratio_df = company_success_launch_df.div(company_launch_df).reset_index()
company_success_launch_ratio_df = company_success_launch_ratio_df.rename(columns= {"Status Mission" : "Success Ratio"})
company_success_launch_ratio_df = company_success_launch_ratio_df.fillna(0)#.sort_values(by = "Success Ratio")
company_success_launch_ratio_df["Launch"] = np.zeros(len(company_success_launch_ratio_df["Company Name"]))
company_success_launch_ratio_df["Success"] = np.zeros(len(company_success_launch_ratio_df["Company Name"]))

for i in range(len(company_success_launch_ratio_df["Company Name"])):
    company_success_launch_ratio_df.loc[i,"Launch"] = company_launch_df[company_success_launch_ratio_df["Company Name"][i]]
    if(company_success_launch_ratio_df["Success Ratio"][i] != 0):
        company_success_launch_ratio_df.loc[i,"Success"] = company_success_launch_df[company_success_launch_ratio_df["Company Name"][i]]
    else:
        company_success_launch_ratio_df.loc[i,"Success"] = 0
company_success_launch_ratio_df.sort_values(by = "Launch").tail(10)


,Company Name,Success Ratio,Launch,Success
8,Northrop,1.000000,12.0,12.0
9,Rocket Lab,0.846154,13.0,11.0
7,MHI,1.000000,14.0,14.0
13,VKS RF,1.000000,17.0,17.0
10,Roscosmos,0.956522,23.0,22.0
5,ISRO,0.958333,24.0,23.0
12,ULA,1.000000,29.0,29.0
0,Arianespace,0.939394,33.0,31.0
11,SpaceX,0.986486,74.0,73.0
1,CASC,0.945055,91.0,86.0


In [120]:
space_company_5_df = spaceLaunch_df[spaceLaunch_df["Year"] > 2015]
space_company_5_df = space_company_5_df[~np.isnan(space_company_5_df["Rocket"])]

company_success_launch_df = space_company_5_df[space_company_5_df["Status Mission"] == "Success"].groupby(["Year","Company Name"])["Status Mission"].count()
company_launch_df = space_company_5_df.groupby(["Year","Company Name"])["Status Mission"].count()

In [130]:
company_success_launch_df[2016]/company_launch_df[2016]

Company Name
Arianespace    1.000000
CASC           0.866667
Eurockot       1.000000
ILS            1.000000
ISRO           1.000000
MHI            1.000000
Northrop       1.000000
Roscosmos      1.000000
SpaceX         0.888889
ULA            1.000000
VKS RF         1.000000
Name: Status Mission, dtype: float64

## Question 2

## Question 3